**NOTA IMPORTANTE**: Las preguntas están marcadas en color rojo. Para la entrega, prepara un documento separado que solo contenga las preguntas y tus respuestas.

<font color="#f00">**La entrega es un documento PDF solo con las preguntas y respuestas**</font>

Las prácticas de este curso las haremos en Python, que tiene dos posibilidades para criptografía: el paquete `PyCryptodome` y el paquete `cryptography`. Ambos son opciones válidas e intercambiables, aunque Las prácticas de este curso las haremos con `PyCryptodome`. Puedes encontrar la ayuda en: https://pycryptodome.readthedocs.io/en/latest/

Si estás siguiendo estas notas en tu PC, puedes instalarlo con: `python3 -m pip install pycryptodome`. Si las estás siguiendo en Colab, ejecuta la siguiente línea:

In [ ]:
# La siguiente línea instala CryptoDome en nuestra máquina virtual
!pip install pycryptodome
# La siguiente línea nos permitirá imprimir líneas largas
from pprint import pprint
from textwrap import wrap

# Cifrado asimétrico o de clave pública

En el cifrado asimétrico o de clave pública, cada participante en la comunicación tiene DOS claves: una de ellas (la clave privada) debe mantenerla totalmente secreta y no decírsela a nadie, mientras que la otra (la clave pública) puede hacerse pública.

Vamos a ver dos ejemplos de estos sistemas:

- Acuerdo de claves Diffiel-Hellman, que permite que dos personas que no se han visto nunca pueda tener una clave AES común
- RSA, que permite hacer tanto cifrado como firma electrónica

## Acuerdo de claves Diffie-Hellman

El protocolo Diffie-Hellman (D-H) permite que dos personas que no se han visto nunca acuerden una clave que después pueden utilizar para el cifrado AES o ChaCha20. Gracias a D-H, Nadie más que esas dos personas sabrán que clave han acordado, incluso si un atacante leyese todos los mensajes que se han intercambiado desde el principio. Esto se conoce como Acuerdo de Intercambio de Claves (https://en.wikipedia.org/wiki/Key_exchange)

Recuerda, el protocolo D-H es:

1. Acuerdan $g$ y $p$ primos entre sí
1. Escogen números en secreto $a$ y $b$
1. Se envian entre ellos:
    - $Alice \rightarrow Bob: A=g^{a} \mod p$
    - $Bob \rightarrow Alice: B=g^{b} \mod p$
1. Calculan en secreto:
    - $Alice$: $s = B^{a} \mod p = g^{ab} \mod p$
    - $Alice$: $s = A^{b} \mod p = g^{ab} \mod p$
1. Y usan $s$ como clave de cifrado un algoritmo simétrico  

A continuación está el código de la librería https://github.com/amiralis/pyDH de Amirali Sanatinia, que es sencillo de leer y entender.

Aunque no parece haber errores evidentes, **es obligatorio utilizar librerías auditadas**. Seguiremos esta por su valor educativo, no porque sea recomendable su uso.

In [ ]:
"""
Pure Python Diffie Hellman implementation

Source: https://github.com/amiralis/pyDH

Apache License
         Version 2.0, January 2004
     Copyright 2015 Amirali Sanatinia
"""

import os
import binascii
import hashlib

# RFC 3526 - More Modular Exponential (MODP) Diffie-Hellman groups for
# Internet Key Exchange (IKE) https://tools.ietf.org/html/rfc3526

primes = {

	# 1536-bit
	5: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA237327FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 2048-bit
	14: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 3072-bit
	15: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A93AD2CAFFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 4096-bit
	16: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C934063199FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 6144-bit
	17: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C93402849236C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BDF8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1BDB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F323A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AACC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE32806A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55CDA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE12BF2D5B0B7474D6E694F91E6DCC4024FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 8192-bit
	18: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C93402849236C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BDF8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1BDB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F323A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AACC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE32806A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55CDA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE12BF2D5B0B7474D6E694F91E6DBE115974A3926F12FEE5E438777CB6A932DF8CD8BEC4D073B931BA3BC832B68D9DD300741FA7BF8AFC47ED2576F6936BA424663AAB639C5AE4F5683423B4742BF1C978238F16CBE39D652DE3FDB8BEFC848AD922222E04A4037C0713EB57A81A23F0C73473FC646CEA306B4BCBC8862F8385DDFA9D4B7FA2C087E879683303ED5BDD3A062B3CF5B3A278A66D2A13F83F44F82DDF310EE074AB6A364597E899A0255DC164F31CC50846851DF9AB48195DED7EA1B1D510BD7EE74D73FAF36BC31ECFA268359046F4EB879F924009438B481C6CD7889A002ED5EE382BC9190DA6FC026E479558E4475677E9AA9E3050E2765694DFC81F56E880B96E7160C980DD98EDD3DFFFFFFFFFFFFFFFFF,
	"generator": 2
	}
}


class DiffieHellman:
	""" Class to represent the Diffie-Hellman key exchange protocol """
	# Current minimum recommendation is 2048 bit.
	def __init__(self, group=14):
		if group in primes:
			self.p = primes[group]["prime"]
			self.g = primes[group]["generator"]
		else:
			raise Exception("Group not supported")

		self.__a = int(binascii.hexlify(os.urandom(32)), base=16)

	def get_private_key(self):
		""" Return the private key (a) """
		return self.__a

	def gen_public_key(self):
		""" Return A, A = g ^ a mod p """
		# calculate G^a mod p
		return pow(self.g, self.__a, self.p)

	def check_other_public_key(self, other_contribution):
		# check if the other public key is valid based on NIST SP800-56
		# 2 <= g^b <= p-2 and Lagrange for safe primes (g^bq)=1, q=(p-1)/2

		if 2 <= other_contribution and other_contribution <= self.p - 2:
			if pow(other_contribution, (self.p - 1) // 2, self.p) == 1:
				return True
		return False

	def gen_shared_key(self, other_contribution):
		""" Return g ^ ab mod p """
		# calculate the shared key G^ab mod p
		if self.check_other_public_key(other_contribution):
			self.shared_key = pow(other_contribution, self.__a, self.p)
			return hashlib.sha256(str(self.shared_key).encode()).digest()
		else:
			raise Exception("Bad public key from other party")


### Alice

En esta ocasión, Alice y Bob quieren acordar una clave que después usarán para las comunicaciones con AES.

Vamos a ver primero qué hace Alice.

In [ ]:
alice = DiffieHellman()
alice_pubkey = alice.gen_public_key()

print('CLAVE PRIVADA DE ALICE: a\n-------------------')
print(f'a = {alice.get_private_key()}\n')

print('\nCLAVE PÚBLICA DE ALICE: es el trío A=(g, p, g^a)\n-------------------')
print(f'g = {alice.g}')
pprint(wrap(f'p = {alice.p}'))
pprint(wrap(f'g^a = {alice_pubkey}'))

Claves de Alice:

- Clave privada es un solo número: $a_{priv}$. Este número es la clave privada de Alice y nunca saldrá del PC de Alice.
- Clave pública es un trío de números: $A_{pub}=\{g, p, g^a\}$. Esta es la clave pública de Alice, que es lo que le envía a Bob.

Nota: en realidad $g$ y $p$ no los suele escoger Alice sino que usa algún valor estándar de una tabla. Así que normalmente solo se envía $g^a$, y los valores de $g$ y $p$ son los del estándar. $g$ suele ser un primo pequeño, 2 ó 3, para acelerar los cálculos, mientras que $p$ es un número muy grande para dar seguridad al sistema.

### Bob

Bob calcular su par de claves públicas y privadas de una forma similar

In [ ]:
bob = DiffieHellman()
bob_pubkey = bob.gen_public_key()

print('CLAVE PRIVADA DE BOB: b\n-------------------')
print(f'b = {bob.get_private_key()}\n')

print('\nCLAVE PÚBLICA DE BOB: es el trío B=(g, p, g^b)\n-------------------')
print(f'g = {bob.g}')
pprint(wrap(f'p = {bob.p}'))
pprint(wrap(f'g^b = {bob_pubkey}'))

### Alice y Bob: cálculo de la clave compartida

Ahora (recuerda que todos estos cálculos son $\mod p$):

- Alice recibe de Bob $B=g^b$, y puede calcular la clave compartida usando su propia $a$: $k_{shared} = B^a=g^{ab}$
- Bob recibe de Alice $A=g^a$, y puede calcular la clave compartida usando su propia $b$: $k_{shared} = A^b=g^{ab}$
- Un espía de las comunicaciones sabe $A=g^a$ y $B=g^b$, pero no sabe $a$ ni $b$. Observa que sabiendo solo $g^a$ y $g^b$ no hay forma de calcular $g^{ab}$ y el atacante no puede calcular la clave

In [ ]:
alice_sharedkey = alice.gen_shared_key(bob_pubkey)
print(f'Clave compartida calculada por Alice: {alice_sharedkey}')
print(f'Tamaño: {len(alice_sharedkey) * 8} bits')

In [ ]:
bob_sharedkey = bob.gen_shared_key(alice_pubkey)
print(f'Clave compartida calculada por Alice: {bob_sharedkey}')
print(f'Tamaño: {len(bob_sharedkey) * 8} bits')

Y podemos comprobar que los dos tienen la misma clave compartida

In [ ]:
print(bob_sharedkey == alice_sharedkey)

<font color="red">
<b>PREGUNTAS:</b>

1. Ya tenemos una "clave compartida", pero aún hay que adaptarla para poder usarla en AES-256 o ChaCha20. ¿Cómo lo harías?
1. Los parámetros p y g de la librería son muy antiguos (RFC3526). ¿Puedes buscar otros más modernos?

</font>

## RSA

Diffie-Hellman solo puede usarse como protocolo de intercambio de clave que después se utiliza en AES.

Veremos a continuación RSA, que ya es un sistema de cifrado asimétrico completo que permite ofrecer confidencialidad y firma electrónica. Dependiendo si ciframos la clave privada o a la inversa, tenemos dos esquemas de funcionamiento:

- **Esquema de cifrado** (confidencialidad): Bob quiere enviar un mensaje a Alice. Bob cifra el mensaje con la clave pública de Bob, y solo Alice puede descifrar ese mensaje con la clave privada de Alice, que solo tiene ella
- **Esquema de firma electrónica** (autenticación): Bob quiere enviar un mensaje a Alice de forma que pueda probar que el mensaje solo lo ha podido enviar Bob. Para ello, cifra el mensaje usando la clave privada de Bob. Cualquiera puede descifrar ese mensaje usando la clave pública de Bob pero, al descifrarlo, saben que el mensaje viene necesariamente de Bob porque es el único que tiene su clave privada.

Puedes obtener confidencialidad y además autenticación ejecutando los dos esquemas a la vez: Bob cifra un mensaje con su clave privada (autenticación) y después con la clave pública de Alice (confidencialidad). De esta manera, el mensaje solo lo puede descifrar Alice y además solo lo ha podido enviar Bob.

Como comentario original, RSA también puede usarse como protocolo de acuerdo de clave, como Diffie-Hellman: una parte simplemente envía a la otra la clave AES que se va a usar, cifrada con RSA.

### Preparación

Antes de empezar necesitaremos un sistema para cálculo del máximo común divisor (mcd en castellano, gcd en inglés) de dos números, y el inverso de un número en un anillo cíclico. Ambas cosas se conocen desde hace tiempo: son dos "algoritmos de Euclides".



**Importante**: <font color="#f00">
no os preocupéis de entender ningún algoritmo o explicación de esta sección de Preparación, simplemente ejecutadlos porque los necesitaremos para el resto de ejercicios.</font>

In [ ]:
# Algoritmo de Euclides para determinar el máximo común divisor (gcd por sus siglas en inglés) de dos enteros a y b
# https://es.wikipedia.org/wiki/M%C3%A1ximo_com%C3%BAn_divisor
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

print('gcd(2, 3) = ', gcd(2, 3))
print('gcd(4, 6) = ', gcd(4, 6))
print('gcd(15, 16) = ', gcd(15, 16))
print('gcd(50720, 48184) = ', gcd(50720, 48184))

# Es decir,
# - 2 y 3 no tienen divisores en común y son co-primos entre ellos
# - 4 y 6 tienen 2 como el máximo divisor común entre ellos
# - 15 y 16 no tienen divisores en común y son co-primos entre ellos. Fíjate que ni 15 ni 16 son primos, decimos que dos números son co-primos si no tienen divisores en común
# - 50720 y 48184 tienen 2536 como el máximo divisor en común

# Inverso multiplicativos de un número en un anillo cíclico $\mathbb{Z}_{\phi}$
# https://es.wikipedia.org/wiki/Anillo_c%C3%ADclico
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi

    while e > 0:
        temp1 = temp_phi // e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2

        x = x2 - temp1 * x1
        y = d - temp1 * y1

        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi
    # no inverse: return None
    return None

print('3^{-1} mod 7 = ', multiplicative_inverse(3, 7))
print('3^{-1} mod 10 = ', multiplicative_inverse(3, 10))
print('2^{-1} mod 10 = ', multiplicative_inverse(2, 10))
print('25^{-1} mod 119 = ', multiplicative_inverse(25, 119))

# Test de si un número es primo
# Algoritmo de testeo muy ineficiente de si un número es primo, pero que nos servirá en nuestros ejercicios porque usaramos números pequeñose.
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5) + 2, 2):
        if num % n == 0:
            return False
    return True

for i in [2, 5, 19, 25, 222, 314, 317]:
    print(f'{i}: ', is_prime(i))

### RSA "manual" con números pequeños

Bien, ya tenemos todo lo necesario para poder hacer los ejercicios.

RSA es un algoritmo de cifrado asimétrico. Es decir, tiene dos claves: una para cifrar y otra diferente para descifrar. Puede usarse tanto para cifrar una información como para firmar digitalmente un documento.

RSA está compuesto por dos funciones sencillas:

- Una función que genera el par de claves necesario. Da como resultado dos claves, una se podrá hacer pública y la otra tiene que permanecer siempre privada.
- Una función que cifra, que es la misma función que para descifrar pero usando la otra clave.

Aquí vemos los dos algoritmos. Fíjate qué sencilla es la función para cifrar o descifrar.

In [ ]:
import random

def rsa_generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p - 1) * (q - 1)

    # Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    # Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)

    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

def rsa_encrypt(pk, number):
    # Unpack the key into it's components
    key, n = pk
    return (number ** key) % n

# The decrypt function is exactly the same than the encrypt function
rsa_decrypt = rsa_encrypt

 En los ejemplos usaremos como parámetros de configuración 17 y 23, dos números primos que simplemente sirven para configurar el algoritmo inicialmente. Cuanto más grandes sean, mayor será el tamaño en bits de la clave. Como estamos usando algoritmos poco eficientes para aprender, no uses números demasiado altos.

In [ ]:
pk, sk = rsa_generate_keypair(17, 23)
print(f'Clave pública pk=(e, n): {pk}')
print(f'Clave privada o secreta sk=(d, n): {sk}')

Fíjate: si generamos otro par de claves, aunque usemos los mismos primos, obtendremos unas claves diferentes. Eso es porque el parámetro $e$ se escoge al azar

In [ ]:
pk, sk = rsa_generate_keypair(17, 23)
print(f'Clave pública pk=(e, n): {pk}')
print(f'Clave privada o secreta sk=(d, n): {sk}')

Vamos a intentar cifrar un texto sencillo:

In [ ]:
print(rsa_encrypt(pk, 'hola'))

No podemos: RSA solo puede cifrar enteros. Una posibilidad es codificar el mensaje como un conjunto de enteros

In [ ]:
msg = [ord(c) for c in 'hola']
print(f'mensaje = {msg}')

c = [rsa_encrypt(pk, m) for m in msg]
print(f'cifrado = {c}')

¿Qué pasa si intentamos cifrar varias veces lo mismo? El texto cifrado es siempre igual. Pocas veces querremos eso. RSA debe usarse siguiendo recomendaciones como PKCS#1. Lo veremos un poco más abajo.

In [ ]:
print([rsa_encrypt(pk, ord(c)) for c in 'aaaa'])

### (semi) Homorfismo

RSA es semihomomórfico con la multiplicación: se pueden hacer cálculos con los números cifrados, aunque no sepas lo que son ni qué resultado tienes. Al descifrar, el resultado es correcto. Más detalles: https://ciberseguridad.com/guias/prevencion-proteccion/criptografia/cifrado-homomorfico/

Por ejemplo, vamos a multiplicar los mensajes cifrados c1 y c2, que son los cifrados de 5 y 2 respectivamente

In [ ]:
m1 = 5
c1 = rsa_encrypt(pk, m1)
print(f'encrypt(pk, {m1}) = {c1}')
print(f'decrypt(sk, {c1}) = {rsa_decrypt(sk, c1)}')
print()

m2 = 2
c2 = rsa_encrypt(pk, m2)
print(f'encrypt(pk, {m2}) = {c2}')
print(f'decrypt(sk, {c2}) = {rsa_decrypt(sk, c2)}')
print()

cm = c1 * c2
print(f"c1 = {c1}; c2 = {c2}; cm = {cm}")

Un atacante no sabe cuánto vale c1 ni c2, ni sabe qué valor tiene cm, pero sabe que, sea lo que sea, ha multiplicado c1 y c2 y cuando se descifre el resultado va a ser correcto

In [ ]:
print(f'decrypt(sk, c1 * c2) = m1 * m2 = {m1} * {m2} = {rsa_decrypt(sk, cm)}')

Según la utilidad, el semihomorfismo puede ser útil o no:

- Sistemas PET (private enhanced technologies) necesitas calcular sin descifrar. Por ejemplo, voto electrónico
- Pero en general no querremos que un atacante pueda multiplicar una orden de pago por otro número y que el resultado sea válido: recomendaciones PKCS#1

### RSA con PyCryptoDome

La función de arriba es para jugar y solo sirve para ver cómo funciona RSA a alto nivel.

Vamos a usar PyCrytoDome, que incluye una librería RSA real.

In [ ]:
from Crypto.PublicKey import RSA

#### Generación de claves con RSA

Mide cuánto tiempo necesitamos para generar las claves ya más útiles de 2048 y 4096 bits. Prueba a generar claves mayores, de 16384 bits, por ejemplo, que es similar en seguridad al cifrado AES de 256 bits.

In [ ]:
# Clave de 2048 bits
key2048 = RSA.generate(2048)
key2048

<font color="red">
<b>PREGUNTAS:</b>

1. Mide cuánto tiempo necesitamos para generar una claves de un tamaño ya útil 2048.
1. Prueba a generar claves mayores, de 4096 bits y de 16384 bits, por ejemplo, que es similar en seguridad al cifrado AES de 256 bits. ¿Cuánto tiempo le lleva aproximadamente?

</font>


#### Cifrado y descifrado con RSA

No es recomendable utilizar RSA "de forma pura", es decir, sin tener en cuenta muchas consideraciones sobre padding, conversiones, longitudes... que se recogen en [PKCS#1](https://en.wikipedia.org/wiki/PKCS_1). De hecho, PyCryptoDome no nos va a dejar utilizar el cifrado y descifrado directamente.

Observa que la línea siguiente resulta en un error, avisando que uses el módulo  `Crypto.Cipher.PKCS1_OAEP` en vez de las funciones directas. PKCS#1 es el estándar de cifrado de RSA y gestiona padding, randomness y otras consideraciones.

In [ ]:
key2048.encrypt(b'hola', None)

**Aunque no se debe**, para entender por qué es necesario PKCS#1 vamos a utilizar la función `_encrypt()` y `_decrypt()`, que no está documentada pero la puedes encontrar en el código: https://github.com/Legrandin/pycryptodome/blob/master/lib/Crypto/PublicKey/RSA.py#L178

Fíjate: vamos simplemente a cifrar el número '15' (que ocuparía 1 byte). Observa que el número '15' cifrado da un número enorme.

In [ ]:
m = 15
c = key2048._encrypt(m)
d = key2048._decrypt(c)

print(f"- Mensaje: {m}")
pprint(wrap(f"- Cifrado: {c}"))
print(f"- Descifrado: {d}")

¿Cómo podemos cifrar una cadena como 'hola'? Pues, por ejemplo, cifrando los bytes de la cadena uno a uno. Observa todo lo que ocuparía esta solución.

In [ ]:
pprint(([wrap(str(key2048._encrypt(c))) for c in b'hola']))

Otra solución es convertir la cadena a un entero con la función: `msg = int.from_bytes(b"hola mundo", "big")`

In [ ]:
msg = b"hola"
msgAsNumber = int.from_bytes(msg, "big")
print(f'Mensaje "{msg}" traducido a número: {msgAsNumber}')

ciphered = key2048._encrypt(msgAsNumber)

pprint(wrap(f'Cifrado: {key2048._encrypt(msgAsNumber)}'))

El problema de esto es que **no podemos cifrar un mensaje más grande que el tamaño en bits de la clave**, es decir, 2048 bits en este caso. Vamos a probar con "hola" * 1000, que es la cadena "hola" repetida 1000 veces

In [ ]:
key2048._encrypt(int.from_bytes(b"hola" * 1000, "big"))

<font color="#f00">

**Pregunta**:
Vamos a hacer las cosas bien: cifra `"hola mundo"` y `"hola mundo" * 1000` usando PKCS1. Encontrarás en ejemplo en la documentación de pyCryptoDome: https://pycryptodome.readthedocs.io/en/latest/src/cipher/oaep.html Usa la clave key2048 que has calculado antes en vez de cargarla desde un archivo
</font>

Vaya, tampoco podemos cifrar un mensaje como `"hola" * 1000` (es decir, 1000 veces la palabra "hola) ni siquiera con PKCS#1, porque el mensaje es más largo que la clave. Esta es una de las razones por las que necesitamos el cifrado híbrido, que veremos a continuación

# Cifrado híbrido

Normalmente no se cifra un mensaje en un sistema de cifrado asimétrico como RSA, sino que se usa algún tipo de cifrado híbrido:

- Cifrar con RSA la clave AES que se usa en las comunicaciones
- Cifrar con RSA el hash de un documento para firma digital

En el tema de TLS veremos un cifrado híbrido: ciframos con RSA la clave AES que usamos para cifrar el texto.

1. Bob: Crea par de claves RSA
1. Alice: Crea clave simétrica AES. Cifra la clave AES con la clave pública de Bob. Envía mensaje
1. Alice: cifra "hola mundo" con clave AES. Envía mensaje
1. Bob: descifra clave AES con clave privada. Descifra mensaje de Alice

Entre los ejemplos de RSA precisamente verás algo así: https://pycryptodome.readthedocs.io/en/latest/src/examples.html#encrypt-data-with-rsa

<font color="red">
<b>Pregunta:</b>

1. ¿Se te ocurre por qué es necesario el cifrado híbrido y no se cifra todo con RSA y ya está?

</font>